In [47]:
from pydantic import BaseModel
from openai import OpenAI
from IPython.display import display, Markdown
import json
import uuid
import os
import base64

from secret_vars import OPENAI_KEY

client = OpenAI(api_key=OPENAI_KEY)

In [49]:
DISPLAY_SPOILERS = True
GENERATE_THEMES_COUNT = 3
GENERATE_RUNES_COUNT = 20
COMFYUI_PATH = "D:\AI\Flux\ComfyUI"

## Theme generation

In [14]:
class RuneTheme(BaseModel):
    name: str
    description: str

class RuneThemes(BaseModel):
    ideas: list[RuneTheme]

def generate_themes() -> RuneThemes:
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": f"I'm making a game in which mundane tasks (such as throwing out trash or doing dishes) can be separated each into multiple runes. What overarching theme can I use for coming up with those runes? Reply with a list of {GENERATE_THEMES_COUNT} propositions."},
        ],
        response_format=RuneThemes,
    )
    return completion.choices[0].message.parsed

def display_themes(themes: RuneThemes):
    if not DISPLAY_SPOILERS:
        return
    display(Markdown("\n".join(f"## {theme.name}\n{theme.description}" for theme in themes.ideas)))

themes = generate_themes()
display_themes(themes)

## Elemental Forces
Each rune represents a different elemental force associated with mundane tasks, such as 'Earth' for grounding or stability in cleaning, 'Water' for washing, and 'Fire' for energy and transformation of items.
## Nature's Cycle
Runes embody stages in nature, representing cycles like growth (planting seeds as a metaphor for beginning a task), decay (throwing out trash), and renewal (cleaning and organizing spaces).
## Celestial Motions
Runes model celestial events or bodies, where each task aligns with cosmic elements—like 'Sun' for illumination (finding motivation), 'Moon' for reflection (organizing thoughts), and 'Stars' for aspirations (setting goals for a clean space).

## Rune generation

In [18]:
class Rune(BaseModel):
    name: str
    description: str

class Runes(BaseModel):
    ideas: list[Rune]

def generate_runes(themes: RuneThemes) -> Runes:
    all_runes: list[Rune] = []
    for i, theme in enumerate(themes.ideas):
        rune_count = round((GENERATE_RUNES_COUNT-len(all_runes))/(len(themes.ideas)-i))
        runes = generate_runes_for_theme(theme, rune_count)
        display_runes(runes, theme)
        all_runes.extend(runes.ideas)
    return Runes(ideas = all_runes)


def generate_runes_for_theme(theme: RuneTheme, count: int) -> Runes:
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": f"I'm making a game in which mundane tasks (such as throwing out trash or doing dishes) can be separated each into multiple runes. Come up with a list of {count} runes that follow the theme \"{theme.name}\" - {theme.description}. The name of each rune should be \"Rune of ...\". Also, for each rune give a brief descriptions what kind of tasks qualify as being \"composed of\" that rune."},
        ],
        response_format=Runes,
    )
    return completion.choices[0].message.parsed

def display_runes(runes: Runes, theme: RuneTheme):
    if not DISPLAY_SPOILERS:
        return
    display(Markdown(f"## {theme.name}:\n" + "\n".join(f" - **{rune.name}**: {rune.description}" for rune in runes.ideas)))

runes = generate_runes(themes)

## Elemental Forces:
 - **Rune of Earth**: Represents grounding and stability, perfect for organizing spaces, decluttering, and creating a structured environment.
 - **Rune of Water**: Symbolizes cleansing and fluidity, associated with washing dishes, laundry, and any tasks that involve water for cleaning.
 - **Rune of Fire**: Embodies transformation and energy, ideal for tasks like cooking, burning waste, or creating warmth and ambiance.
 - **Rune of Air**: Connotes freshness and movement, connected to tasks involving ventilation, dusting, and refreshing spaces with air circulation.
 - **Rune of Metal**: Represents strength and preservation, suited for tasks involving repairing, recycling metallic items, and maintaining tools.
 - **Rune of Wood**: Embodies growth and nourishment, relevant to gardening, organizing wooden items, and caring for houseplants.
 - **Rune of Spirit**: Symbolizes motivation and intention, tied to planning tasks, goal-setting, and infusing positivity into mundane chores.

## Nature's Cycle:
 - **Rune of Seedling**: Symbolizes the beginning of a task. Tasks may include brainstorming ideas, making a to-do list, or multiplying the motivation to start a project, similar to planting seeds for future growth.
 - **Rune of Growth**: Embodies the nurturing stage where tasks are developed and expanded. Tasks like gathering materials, practicing skills, or gradually working on a project reflect this increasable growth phase.
 - **Rune of Bloom**: Represents the flourishing culmination of a task. Completing stages of a project, showing off your work, or celebrating a successfully finished chore embody this vibrant expression of effort.
 - **Rune of Wither**: Symbolizes decay and the importance of letting go. Tasks such as decluttering, throwing away expired food, or getting rid of old clothes highlight the necessity of clearing away the old to make space for the new.
 - **Rune of Rejuvenation**: Embodies revitalization and restoration. Tasks like cleaning, organizing, and refreshing spaces signify the act of re-energizing environments, akin to the earth rejuvenating after a season.
 - **Rune of Renewal**: Represents the cycle of revival and transformation. Engaging in recycling, repurposing old items, or recharging spiritually through meditation and self-care aligns with the themes of nature's continual renewal process.

## Celestial Motions:
 - **Rune of the Sun**: Represents illumination and energy. Tasks that fall under this rune involve finding motivation and brightening your space. These can include vacuuming, dusting, or organizing items to make the environment feel more vibrant and lively.
 - **Rune of the Moon**: Symbolizes reflection and contemplation. Tasks composed of this rune focus on introspection and organizing one's thoughts. Suitable tasks could be writing in a journal, planning out your week, or decluttering one’s mind by engaging in meditation.
 - **Rune of the Stars**: Represents aspirations and dreams. This rune embodies setting goals for your clean space, such as creating a cleaning schedule, designing a layout for better organization, or establishing a list of improvements you want to make in your living area.
 - **Rune of the Comet**: Symbolizes swift change and transformation. Tasks that embody this rune include quick tidying up, throwing out expired food, or making instant improvements like rearranging furniture for a more functional space.
 - **Rune of the Galaxy**: Represents a sense of community and shared space. Tasks influenced by this rune involve collaboration, such as inviting roommates to participate in a group clean-up, organizing a communal dinner night, or setting up a chore rotation.
 - **Rune of the Nebula**: Symbolizes creativity and birth of ideas. Tasks associated with this rune can include redecorating a room, upcycling furniture, or brainstorming new organizing solutions for merging aesthetics with functionality.
 - **Rune of the Eclipse**: Represents the balance between light and dark, symbolizing cleansing and renewal. Tasks under this rune could be deeply cleansing, such as deep cleaning an area, purging items you no longer use, or reorganizing after a big life change.

## Generate rune sprites

In [57]:
def generate_rune_sprites(runes: list[str]):
    path_here = os.popen('powershell.exe -Command "(Get-Item .).FullName"').read().replace("\n", "")
    uid = uuid.uuid4()
    runes_str = json.dumps(runes).replace('"', '\\"')

    command = f'cd {COMFYUI_PATH}; python "{path_here}\image_gen_runes.py" --uuid "{uid}" --runes \'{runes_str}\''
    full_command = f'powershell.exe -Command {json.dumps(command)}'
    os.system(full_command)

    commands = []
    for rune in runes:
        rune_encoded = base64.b64encode(rune.encode("ascii")).decode("ascii")
        for i in range(1, 5):
            command = f'Move-Item -Path "{COMFYUI_PATH}\\output\\rune_{uid}_{rune}_0000{i}_.png" -Destination "{path_here}\\img\\{rune_encoded}_{i}.png" -Force'
            commands.append(command)
    print("; ".join(commands))
    os.system(f'powershell.exe -Command {json.dumps("; ".join(commands))}')

generate_rune_sprites(["the Eclipse", "Seedling", "the Comet"])

Total VRAM 12282 MB, total RAM 32609 MB
pytorch version: 2.5.0+cu124
WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.0.1+cu118 with CUDA 1108 (you have 2.5.0+cu124)
    Python  3.10.11 (you have 3.10.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
C:\Users\Aurif\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xformers\ops\swiglu_op.py:107: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(cls, ctx, x, w1, b1, w2, b2, w3, b3):
C:\Users\Aurif\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xformers\ops\swiglu_op.py:

ComfyUI found: D:\AI\Flux\ComfyUI
'D:\AI\Flux\ComfyUI' added to sys.path
Could not find the extra_model_paths config file.
ComfyUI found: D:\AI\Flux\ComfyUI
'D:\AI\Flux\ComfyUI' added to sys.path


100%|##########| 15/15 [00:16<00:00,  1.13s/it]
Requested to load AutoencodingEngine
Loading 1 new model
loaded completely 0.0 159.87335777282715 True
Requested to load FluxClipModel_
Loading 1 new model
loaded completely 0.0 4777.53759765625 True
loaded partially 9545.67451171875 9545.5224609375 0
100%|##########| 15/15 [00:14<00:00,  1.05it/s]
Requested to load AutoencodingEngine
Loading 1 new model
loaded completely 0.0 159.87335777282715 True
Requested to load FluxClipModel_
Loading 1 new model
loaded completely 0.0 4777.53759765625 True
loaded partially 9545.67451171875 9545.5224609375 0
100%|##########| 15/15 [00:14<00:00,  1.04it/s]
Requested to load AutoencodingEngine
Loading 1 new model
loaded completely 0.0 159.87335777282715 True


Move-Item -Path "D:\AI\Flux\ComfyUI\output\rune_88e26111-3dd6-4327-b94d-1f0ec17ffa9d_the Eclipse_00001_.png" -Destination "D:\Projects\Az_zy - idle\Generation\img\dGhlIEVjbGlwc2U=_1.png" -Force; Move-Item -Path "D:\AI\Flux\ComfyUI\output\rune_88e26111-3dd6-4327-b94d-1f0ec17ffa9d_the Eclipse_00002_.png" -Destination "D:\Projects\Az_zy - idle\Generation\img\dGhlIEVjbGlwc2U=_2.png" -Force; Move-Item -Path "D:\AI\Flux\ComfyUI\output\rune_88e26111-3dd6-4327-b94d-1f0ec17ffa9d_the Eclipse_00003_.png" -Destination "D:\Projects\Az_zy - idle\Generation\img\dGhlIEVjbGlwc2U=_3.png" -Force; Move-Item -Path "D:\AI\Flux\ComfyUI\output\rune_88e26111-3dd6-4327-b94d-1f0ec17ffa9d_the Eclipse_00004_.png" -Destination "D:\Projects\Az_zy - idle\Generation\img\dGhlIEVjbGlwc2U=_4.png" -Force; Move-Item -Path "D:\AI\Flux\ComfyUI\output\rune_88e26111-3dd6-4327-b94d-1f0ec17ffa9d_Seedling_00001_.png" -Destination "D:\Projects\Az_zy - idle\Generation\img\U2VlZGxpbmc=_1.png" -Force; Move-Item -Path "D:\AI\Flux\Comf

In [55]:
base64.b64encode('abc'.encode("ascii")).decode("ascii")

'YWJj'